In [1]:
!pip install ipython==7.34.0 ipykernel==5.5.6
!pip install import_ipynb

In [ ]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split

from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/Colab Notebooks/Physics-Informed Neural Networks/Demo/fenics_cfd/neural_network'
os.chdir(path)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device {device}')
if device.type == 'cuda':
  print(torch.cuda.get_device_name())
  torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

import import_ipynb
from data_processing import FlowFieldDataset, train_test_split
from nn_modules import SuperResolutionCNN
from model_utils import train_model, test_model, evaluate_model

In [3]:
def relative_error_loss(output, target):
  epsilon = 1e-6
  loss = torch.mean(torch.abs((output - target) / (target + epsilon)))
  return loss

In [4]:
npy_velocity_dir = '/content/drive/MyDrive/Colab Notebooks/Physics-Informed Neural Networks/Demo/fenics_cfd/blood_flow_simulations/velocity_fields/npy_data'
npy_pressure_dir = '/content/drive/MyDrive/Colab Notebooks/Physics-Informed Neural Networks/Demo/fenics_cfd/blood_flow_simulations/pressure_fields/npy_data'

dataset = FlowFieldDataset(npy_velocity_dir, npy_pressure_dir)
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2)

In [8]:
batch_size = 1
num_epochs = 1

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

hidden_layers = np.array([16])
model = SuperResolutionCNN(hidden_layers).to(device)
criterion = torch.nn.MSELoss()
# criterion = relative_error_loss
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

data_losses = train_model(model, train_loader, optimizer, criterion, num_epochs, device)

Epoch [1/1], Step [10/1658], Loss: 0.130119
Epoch [1/1], Step [20/1658], Loss: 0.292098


KeyboardInterrupt: ignored

In [ ]:
avg_loss, results = test_model(model, test_loader, criterion, device, num_visualizations=3)

In [ ]:
evaluate_model(model, test_loader, device)